Imports

In [1]:
import feed_forward
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import geoopt
from time import time
from torchvision import datasets, transforms
torch.cuda.is_available()

True

CUDA check

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))])

In [4]:
trainset = datasets.CIFAR10('PATH_TO_STORE_TRAINSET', download=True, train=True, transform=transform)
valset = datasets.CIFAR10('PATH_TO_STORE_TESTSET', download=True, train=False, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)
# valloader = torch.utils.data.DataLoader(valset, batch_size=512, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label]) for img, label in trainset if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in valset if label in [0, 2]]

Initializing the model

In [6]:
model = feed_forward.HypFF(3072, 512, 256, 2)
model

HypFF(
  (fc1): MobLinear(
    in_features=3072, out_features=512, bias=True
    (ball): PoincareBall manifold
  )
  (fc2): MobLinear(
    in_features=512, out_features=256, bias=True
    (ball): PoincareBall manifold
  )
  (fc3): MobLinear(
    in_features=256, out_features=2, bias=True
    (ball): PoincareBall manifold
  )
)

In [7]:
learning_rate = 1e-2

In [8]:
optimizer = geoopt.optim.RiemannianSGD(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()

In [9]:
n_epochs = 10
for epoch in range(n_epochs):
    for img, label in cifar2:
        # img, label = img.to(device), label.to(device)
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

KeyboardInterrupt: 